In [ ]:
library(Seurat)
library(ape)
library(ggtree)
library(ggplot2)
library(dplyr)
library(tidyverse)


In [ ]:
#mel<-readRDS("skin_atlas/Melanoma_only_sc_reproduced_final_oct31.rds")
mel <- read.delim("/QRISdata/Q2051/SCC_Paper/resources/data/frozen_objects_Nov2024_PP/Mel/Mel_full_object_final_Nov26_metadata.txt", sep = "\t", stringsAsFactors = FALSE)
df<-data.frame(level1=mel$Level1, level2=mel$Level2,  level3=mel$Level3_Cancer)
# Count the occurrences for each combination
counts <- df %>% group_by(level1, level2, level3) %>% tally()
counts$level1 <- gsub(" ", "", counts$level1)
counts$level2 <- gsub(" ", "", counts$level2)
counts$level3 <- gsub(" ", "", counts$level3)
#counts

#counts<-counts[-c(11,18,20),]
counts$level2<-gsub("cell","Cell",counts$level2)
# Function to create Newick string including Level3
create_newick_str <- function(df) {
  level1_nodes <- unique(df$level1)
  newick_str <- ""
  
  for (node in level1_nodes) {
    level2_nodes <- df %>% filter(level1 == node) %>% pull(level2) %>% unique()
    level2_str <- ""
    
    for (level2_node in level2_nodes) {
      children <- df %>% filter(level1 == node, level2 == level2_node) %>% pull(level3)
      children_str <- paste(children, collapse = ",")
      level2_str <- paste0(level2_str, "(", children_str, ")", level2_node, ",")
    }
    
    level2_str <- substr(level2_str, 1, nchar(level2_str) - 1)  # Remove trailing comma
    newick_str <- paste0(newick_str, "(", level2_str, ")", node, ",")
  }
  
  newick_str <- substr(newick_str, 1, nchar(newick_str) - 1)  # Remove trailing comma
  newick_str <- paste0("(", newick_str, ");")
  return(newick_str)
}

# Generate the Newick string
newick_str <- create_newick_str(counts)
print(newick_str)

# Create the tree from the Newick string
tree <- read.tree(text = newick_str)

# Add counts to the data
counts$counts <- counts$n

# Plot the tree
p <- ggtree(tree, layout="dendrogram") +
  geom_tiplab(aes(label=label), hjust=-0.3)

# Extract the data from the ggtree object
tree_data <- p$data

# Merge counts with tree data by matching level3 to label in tree_data
tree_data <- tree_data %>%
  left_join(counts, by = c("label" = "level3"), relationship = "many-to-many")

# Ensure counts is not NA
tree_data$counts[is.na(tree_data$counts)] <- 1

# Define color list based on cell types
color_list <-c("DC"= "#5f9d9e",
    "EndothelialCell"= "#f8a41e",
    "Fibroblast"= "#458b41",
    "KCBasal"= "#f16b6b",
    "KCCornified"= "#9a1f61",
    "KCGranular"= "#c72685",
    "KCDifferentiating"= "#9583bd",
    "KCHair"= "#eb2627",
    "LC"= "#37479b",
    "Macrophage"= "#eae71d",
    "Melanocytes"= "#8b471f",
    "Melanoma"= "black",
    "NK"= "#99ca3e",
    "TCell"= "#41baeb",
    "Treg"= "#bbe5f3",
    "pDC"= "#66bf76",
    "MastCell"= "#7f8133",
    "mRegDC"="#105146",
    'CD8+TCell'= "#406573",
    "BCell"="#fed9b9",
    "Pericytes"="#dca566",
    "Sweatglandrelated"="#f2634b",
    "nan"="grey",
    'SchwannCell'='#0b507b'
)
# Assign colors only to the matching labels
tree_data <- tree_data %>%
  mutate(color = color_list[tree_data$label])

# Plot the tree horizontally with counts and colors
options(repr.plot.width = 15, repr.plot.height = 2)

x_min <- min(tree_data$x, na.rm = TRUE)
x_max <- max(tree_data$x, na.rm = TRUE)

#pdf("scc_tree_3levels_imm.pdf", width=15, height = 2)
ggtree(tree, layout="dendrogram") +
  geom_tiplab(aes(label=label), hjust=10) +
  geom_point(data=tree_data, aes(x=x, y=y, size=counts, fill=label), shape=21, color="black") +
  theme_tree2() +
  scale_size_continuous(range = c(3, 10), limits = c(1, max(tree_data$counts, na.rm = TRUE))) + # Adjust size range and limits
  coord_flip() +  # Flip coordinates to make the tree horizontal
  xlim(x_min - 1, x_max + 1) + # Expand x-axis limits
  theme(axis.text.x = element_blank(),  # Remove x-axis text
        axis.ticks.x = element_blank(), # Remove x-axis ticks
        axis.title.x = element_blank(), # Remove x-axis title
        axis.line.x = element_blank()) + # Remove x-axis line
  scale_fill_manual(values=color_list)
#dev.off()

In [ ]:
pdf("/QRISdata/Q2051/SCC_Paper/resources/data/reanalysis_figs/mel/updated/Mel_tree_new_Level3_cancer.pdf",width = 15,height=2)
ggtree(tree, layout="dendrogram") +
  geom_tiplab(aes(label=label), hjust=10) +
  geom_point(data=tree_data, aes(x=x, y=y, size=counts, fill=label), shape=21, color="black") +
  theme_tree2() +
  scale_size_continuous(range = c(3, 10), limits = c(1, max(tree_data$counts, na.rm = TRUE))) + # Adjust size range and limits
  coord_flip() +  # Flip coordinates to make the tree horizontal
  xlim(x_min - 1, x_max + 1) + # Expand x-axis limits
  theme(axis.text.x = element_blank(),  # Remove x-axis text
        axis.ticks.x = element_blank(), # Remove x-axis ticks
        axis.title.x = element_blank(), # Remove x-axis title
        axis.line.x = element_blank()) + # Remove x-axis line
  scale_fill_manual(values=color_list)
dev.off()

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
#pdf("/QRISdata/Q2051/SCC_Paper/resources/data/reanalysis_figs/mel/updated/Mel_tree_labels_for_ref_since_cut.pdf",width = 8,height=8)
ggtree(tree, layout="rectangular") +
  geom_tiplab(aes(label=label), hjust=1) + # Adjust horizontal justification for labels
  geom_point(data=tree_data, aes(x=x, y=y, size=counts, fill=label), shape=21, color="black") +
  theme_tree2() +
  scale_size_continuous(range = c(3, 10), limits = c(1, max(tree_data$counts, na.rm = TRUE))) + # Adjust size range and limits
  theme(axis.text.y = element_blank(),  # Remove y-axis text
        axis.ticks.y = element_blank(), # Remove y-axis ticks
        axis.title.y = element_blank(), # Remove y-axis title
        axis.line.y = element_blank()) + # Remove y-axis line
  scale_fill_manual(values=color_list) 
#dev.off()